In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [2]:
# Load the CLEANED data, not the raw one
df = pd.read_csv('data/cleaned_resume_data.csv')

# Verify it has the column we need
print(df.columns) 
# Should show 'Resume', 'Category', and 'cleaned_resume'

Index(['Category', 'Resume', 'cleaned_resume'], dtype='str')


In [3]:
# --- 1. Label Encoding ---
# The computer doesn't understand "Data Science", it only understands numbers.
le = LabelEncoder()
df['Category_ID'] = le.fit_transform(df['Category'])

In [4]:
df.head()

,Category,Resume,cleaned_resume,Category_ID
0,Data Science,Skills * Programming Languages: Python (pandas...,skills programming languages python pandas num...,6
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education details may 2013 to may 2017 b e uit...,6
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",areas of interest deep learning control system...,6
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skills r python sap hana tableau sap hana sql ...,6
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education details mca ymcaust faridabad haryan...,6


In [5]:
# --- 2. Vectorization (TF-IDF) ---
# This converts words to numbers based on frequency.
tfidf = TfidfVectorizer(stop_words='english', max_features=1000) 
# max_features=1000 means we only care about the top 1000 most common words to keep it fast.

required_text = df['cleaned_resume'].values #Take only the cleaned resume column and convert it to a numpy array
required_target = df['Category_ID'].values  # Convert the text data into numerical vectors 

word_vector = tfidf.fit_transform(required_text)

In [6]:
print(word_vector.shape)
print(len(required_target))


(962, 1000)
962


In [7]:
# --- 3. Split Data ---
# 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(
    word_vector, required_target, random_state=42, test_size=0.2
)

In [8]:
# --- 4. Model Training ---
# We use OneVsRest because this is a Multi-Class problem.
# It treats each category as a separate "Yes/No" problem.
clf = OneVsRestClassifier(KNeighborsClassifier())
clf.fit(X_train, y_train)

,"estimator estimator: estimator objectA regressor or a classifier that implements :term:`fit`.When a classifier is passed, :term:`decision_function` will be usedin priority and it will fallback to :term:`predict_proba` if it is notavailable.When a regressor is passed, :term:`predict` is used.",KNeighborsClassifier()
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation: the `n_classes`one-vs-rest problems are computed in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: 0.20 `n_jobs` default changed from 1 to None",None
,"verbose verbose: int, default=0The verbosity level, if non zero, progress messages are printed.Below 50, the output is sent to stderr. Otherwise, the output is sentto stdout. The frequency of the messages increases with the verbositylevel, reporting all iterations at 10. See :class:`joblib.Parallel` formore details... versionadded:: 1.1",0
,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",5
,"weights weights: {'uniform', 'distance'}, callable or None, default='uniform'Weight function used in prediction. Possible values:- 'uniform' : uniform weights. All points in each neighborhood are weighted equally.- 'distance' : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.- [callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.Refer to the example entitled:ref:`sphx_glr_auto_examples_neighbors_plot_classification.py`showing the impact of the `weights` parameter on the decisionboundary.",'uniform'
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"p p: float, default=2Power parameter for the Minkowski metric. When p = 1, this is equivalentto using manhattan_distance (l1), and euclidean_distance (l2) for p = 2.For arbitrary p, minkowski_distance (l_p) is used. This parameter is expectedto be positive.",2
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'minkowski'
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.Doesn't affect :meth:`fit` method.",None


In [9]:
# --- 5. Evaluation ---
prediction = clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, prediction):.2f}")
print("\nClassification Report:\n")
print(classification_report(y_test, prediction))

Accuracy: 0.98

Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         7
           4       1.00      1.00      1.00         4
           5       1.00      1.00      1.00         9
           6       0.75      0.60      0.67         5
           7       1.00      0.88      0.93         8
           8       1.00      0.93      0.96        14
           9       1.00      1.00      1.00         5
          10       1.00      1.00      1.00         7
          11       1.00      1.00      1.00         6
          12       1.00      1.00      1.00        12
          13       1.00      1.00      1.00         4
          14       1.00      1.00      1.00         7
          15       1.00      1.00      1.00        15
          16       1.00      1.00      1.

In [10]:
# --- 6. Save the Artifacts (CRITICAL STEP) ---
# We need to save the 'tfidf' (to process new resumes) and 'clf' (to predict).
import os

# Create the artifacts directory if it doesn't exist
os.makedirs('artifacts', exist_ok=True)

pickle.dump(tfidf, open('artifacts/tfidf.pkl', 'wb'))
pickle.dump(clf, open('artifacts/model.pkl', 'wb'))
# We also need to save the Label Encoder to decode the numbers back to names later!
pickle.dump(le, open('artifacts/label_encoder.pkl', 'wb'))

print("Artifacts saved to 'artifacts/' folder.")

Artifacts saved to 'artifacts/' folder.
